# 🔴 Brief 3 : Refactoring et mise en qualité d'un code legacy

**Badge:** 🔴 Avancé  
**Durée:** 3 heures  
**Sections validées:** 05-Qualité-Tests

---

## 📋 Contexte

Vous venez de rejoindre une équipe de data engineering. Votre première tâche : reprendre un script Python écrit il y a 2 ans par un stagiaire qui a depuis quitté l'entreprise.

Le script fonctionne... mais :
- ❌ Aucun type hint
- ❌ Aucune docstring
- ❌ Noms de variables incompréhensibles (x, y, tmp, data2)
- ❌ Aucune gestion d'erreurs
- ❌ Magic numbers partout
- ❌ Une seule fonction gigantesque (200 lignes)
- ❌ Aucun test
- ❌ Aucune configuration (ruff, pre-commit)

**Votre mission :** Transformer ce code "sale" en code professionnel, maintenable et testé.

---

## 🎯 Objectifs du projet

Refactorer le code legacy pour :

1. ✅ Ajouter des type hints sur toutes les fonctions
2. ✅ Ajouter des docstrings (style Google)
3. ✅ Refactorer en fonctions/classes bien découpées
4. ✅ Renommer les variables avec des noms explicites
5. ✅ Ajouter la gestion d'erreurs (try/except, exceptions custom)
6. ✅ Écrire des tests pytest (coverage > 80%)
7. ✅ Configurer ruff (linter/formatter)
8. ✅ Configurer pre-commit (hooks Git)
9. ✅ Ajouter du logging structuré
10. ✅ Profiler et optimiser si possible

---

## 📝 Le code "sale" à refactorer

Voici le code legacy que vous devez améliorer.

In [ ]:
%%writefile legacy_code.py
import json
import csv

def process(f, f2, t):
    data = []
    if t == 'csv':
        x = open(f, 'r')
        r = csv.DictReader(x)
        for row in r:
            data.append(row)
        x.close()
    else:
        x = open(f, 'r')
        data = json.load(x)
        x.close()
    
    # nettoyage
    data2 = []
    for d in data:
        tmp = {}
        for k, v in d.items():
            if v != '' and v != None:
                if k == 'age':
                    tmp[k] = int(v)
                elif k == 'salary':
                    tmp[k] = float(v)
                else:
                    tmp[k] = v.strip() if isinstance(v, str) else v
        if len(tmp) > 0:
            data2.append(tmp)
    
    # stats
    total = len(data2)
    ages = [d['age'] for d in data2 if 'age' in d]
    avg_age = sum(ages) / len(ages)
    
    salaries = [d['salary'] for d in data2 if 'salary' in d]
    avg_salary = sum(salaries) / len(salaries)
    
    # groupby city
    cities = {}
    for d in data2:
        c = d.get('city', 'Unknown')
        if c not in cities:
            cities[c] = []
        cities[c].append(d)
    
    # tri par salaire
    sorted_data = sorted(data2, key=lambda x: x.get('salary', 0), reverse=True)
    
    # filtrage senior
    seniors = [d for d in data2 if d.get('age', 0) >= 50]
    
    # export
    result = {
        'total': total,
        'avg_age': avg_age,
        'avg_salary': avg_salary,
        'by_city': {k: len(v) for k, v in cities.items()},
        'top_earners': sorted_data[:5],
        'seniors': seniors
    }
    
    y = open(f2, 'w')
    json.dump(result, y, indent=2)
    y.close()
    
    print('Done!')
    return result


if __name__ == '__main__':
    process('employees.csv', 'report.json', 'csv')

---

## 📝 Fichier de test pour le code legacy

In [ ]:
%%writefile employees.csv
name,age,city,salary,department
Alice Johnson,28,Paris,55000.50,Engineering
  Bob Smith  ,35,Lyon,62000,Sales
Charlie Brown,,Marseille,48000,Marketing
David Lee,52,Paris,75000.75,Engineering
Eve Davis,45,Lyon,,HR
Frank Miller,58,Paris,82000,Engineering
Grace Wilson,31,Marseille,51000,Sales
Henry Taylor,41,Lyon,64000,Engineering
Iris Anderson,29,,49000,Marketing
Jack Thomas,54,Paris,78000,Engineering

---

## 📋 Spécifications techniques

### 1. Type hints

Ajouter des type hints partout :
- Paramètres de fonctions
- Valeurs de retour
- Variables complexes (List, Dict, Optional, etc.)

```python
from typing import List, Dict, Optional, Any

def read_csv_file(file_path: str) -> List[Dict[str, Any]]:
    ...
```

### 2. Docstrings (style Google)

```python
def calculate_average_age(employees: List[Dict[str, Any]]) -> float:
    """Calcule l'âge moyen des employés.
    
    Args:
        employees: Liste de dictionnaires représentant les employés.
        
    Returns:
        L'âge moyen en tant que flottant.
        
    Raises:
        ValueError: Si la liste est vide ou si aucun employé n'a d'âge.
    """
    ...
```

### 3. Refactoring

Découper la fonction géante en fonctions spécialisées :
- `read_csv_file(file_path: str) -> List[Dict]`
- `read_json_file(file_path: str) -> List[Dict]`
- `clean_employee_data(data: List[Dict]) -> List[Dict]`
- `calculate_statistics(employees: List[Dict]) -> Dict`
- `group_by_city(employees: List[Dict]) -> Dict[str, List[Dict]]`
- `filter_seniors(employees: List[Dict], age_threshold: int) -> List[Dict]`
- `export_report(report: Dict, output_path: str) -> None`

Optionnel : Créer une classe `EmployeeAnalyzer` pour regrouper la logique.

### 4. Gestion d'erreurs

Gérer les erreurs avec des exceptions custom :

```python
class EmployeeDataError(Exception):
    """Exception de base pour les erreurs de données."""
    pass

class FileReadError(EmployeeDataError):
    """Levée quand un fichier ne peut pas être lu."""
    pass

class InvalidDataError(EmployeeDataError):
    """Levée quand les données sont invalides."""
    pass
```

### 5. Tests pytest

Écrire des tests pour couvrir :
- Lecture de fichiers CSV/JSON
- Nettoyage des données
- Calculs statistiques
- Groupement par ville
- Filtrage des seniors
- Gestion d'erreurs

**Objectif : Coverage > 80%**

```bash
pytest --cov=employee_analyzer --cov-report=html
```

### 6. Configuration ruff

Créer un fichier `pyproject.toml` avec la configuration ruff.

### 7. Configuration pre-commit

Créer un fichier `.pre-commit-config.yaml` avec les hooks :
- ruff (linter)
- ruff-format (formatter)
- mypy (type checker)

### 8. Logging

Remplacer les `print()` par du logging structuré :

```python
import logging

logger = logging.getLogger(__name__)

logger.info("Processing %d employees", len(employees))
logger.debug("Average age calculated: %.2f", avg_age)
logger.error("Failed to read file: %s", file_path)
```

### 9. Optimisation (bonus)

Profiler le code avec `cProfile` ou `line_profiler` et optimiser si nécessaire.

---

## 📦 Livrables

### Checklist

- [ ] Code refactoré dans `employee_analyzer.py`
- [ ] Type hints sur toutes les fonctions
- [ ] Docstrings (style Google) partout
- [ ] Fonctions bien découpées (< 30 lignes par fonction)
- [ ] Noms de variables explicites
- [ ] Constantes nommées (pas de magic numbers)
- [ ] Exceptions custom définies et utilisées
- [ ] Fichier `test_employee_analyzer.py` avec 8+ tests
- [ ] Coverage pytest > 80%
- [ ] Fichier `pyproject.toml` avec config ruff
- [ ] Fichier `.pre-commit-config.yaml`
- [ ] Logging configuré et utilisé
- [ ] Code passe ruff sans erreur
- [ ] Code passe mypy sans erreur
- [ ] README.md avec instructions

---

## 🎯 Grille d'évaluation

| Compétence | Critères d'évaluation | Points |
|------------|----------------------|--------|
| **Type hints** | Complets sur toutes fonctions, types précis (List, Dict, Optional) | **/15** |
| **Refactoring** | Fonctions bien découpées, nommage clair, structure logique | **/20** |
| **Tests pytest** | 8+ tests, coverage > 80%, cas limites couverts | **/20** |
| **Config ruff + pre-commit** | Fichiers corrects, hooks fonctionnels, code conforme | **/15** |
| **Gestion erreurs** | Exceptions custom, try/except pertinents | **/10** |
| **Logging** | Logger configuré, niveaux appropriés, messages clairs | **/10** |
| **Documentation** | Docstrings complètes, README, commentaires utiles | **/10** |
| **TOTAL** | | **/100** |

### Critères de réussite

- ✅ **Acquis (≥ 70/100)** : Code professionnel, testé, conforme
- 🚧 **En cours d'acquisition (50-69/100)** : Refactoring partiel
- ❌ **Non acquis (< 50/100)** : Code toujours de mauvaise qualité

---

## 💡 Template pyproject.toml

In [ ]:
%%writefile pyproject.toml
[project]
name = "employee-analyzer"
version = "1.0.0"
description = "Analyse de données employés (refactoré)"
requires-python = ">=3.11"
dependencies = [
    "pytest>=7.4.0",
    "pytest-cov>=4.1.0",
]

[tool.ruff]
line-length = 100
target-version = "py311"

[tool.ruff.lint]
select = [
    "E",   # pycodestyle errors
    "W",   # pycodestyle warnings
    "F",   # pyflakes
    "I",   # isort
    "B",   # flake8-bugbear
    "C4",  # flake8-comprehensions
    "UP",  # pyupgrade
]
ignore = []

[tool.ruff.lint.per-file-ignores]
"__init__.py" = ["F401"]  # Ignore unused imports in __init__.py

[tool.pytest.ini_options]
testpaths = ["tests"]
python_files = "test_*.py"
addopts = "-v --cov=employee_analyzer --cov-report=term-missing"

[tool.mypy]
python_version = "3.11"
warn_return_any = true
warn_unused_configs = true
disallow_untyped_defs = true

---

## 💡 Template .pre-commit-config.yaml

In [ ]:
%%writefile .pre-commit-config.yaml
repos:
  - repo: https://github.com/astral-sh/ruff-pre-commit
    rev: v0.4.0
    hooks:
      # Linter
      - id: ruff
        args: [--fix]
      # Formatter
      - id: ruff-format

  - repo: https://github.com/pre-commit/mirrors-mypy
    rev: v1.10.0
    hooks:
      - id: mypy
        additional_dependencies: [types-all]

  - repo: https://github.com/pre-commit/pre-commit-hooks
    rev: v4.5.0
    hooks:
      - id: trailing-whitespace
      - id: end-of-file-fixer
      - id: check-yaml
      - id: check-json
      - id: check-added-large-files

---

## 💡 Exemple de structure refactorée

Voici une suggestion de structure (ne PAS copier-coller, juste pour vous guider) :

In [ ]:
# employee_analyzer.py - STRUCTURE SEULEMENT (à compléter)

import csv
import json
import logging
from pathlib import Path
from typing import Any, Dict, List, Optional

# Configuration logging
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] [%(levelname)s] %(message)s",
)
logger = logging.getLogger(__name__)

# Constantes
SENIOR_AGE_THRESHOLD = 50
TOP_EARNERS_LIMIT = 5
DEFAULT_CITY = "Unknown"


# Exceptions
class EmployeeDataError(Exception):
    """Exception de base pour les erreurs de données employés."""
    pass

# TODO: Ajouter FileReadError, InvalidDataError


# Fonctions de lecture
def read_csv_file(file_path: str) -> List[Dict[str, Any]]:
    """Lit un fichier CSV et retourne une liste de dictionnaires.
    
    Args:
        file_path: Chemin vers le fichier CSV.
        
    Returns:
        Liste de dictionnaires représentant les employés.
        
    Raises:
        FileReadError: Si le fichier ne peut pas être lu.
    """
    # TODO: Implémenter avec gestion d'erreurs
    pass


def read_json_file(file_path: str) -> List[Dict[str, Any]]:
    """Lit un fichier JSON et retourne une liste de dictionnaires."""
    # TODO: Implémenter
    pass


# Nettoyage
def clean_employee_data(employees: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """Nettoie les données employés (supprime valeurs vides, convertit types)."""
    # TODO: Implémenter
    pass


def clean_employee_record(record: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """Nettoie un enregistrement employé individuel."""
    # TODO: Implémenter
    pass


# Statistiques
def calculate_average_age(employees: List[Dict[str, Any]]) -> float:
    """Calcule l'âge moyen des employés."""
    # TODO: Implémenter avec gestion ZeroDivisionError
    pass


def calculate_average_salary(employees: List[Dict[str, Any]]) -> float:
    """Calcule le salaire moyen des employés."""
    # TODO: Implémenter
    pass


def calculate_statistics(employees: List[Dict[str, Any]]) -> Dict[str, Any]:
    """Calcule toutes les statistiques sur les employés."""
    # TODO: Implémenter
    pass


# Groupement et filtrage
def group_by_city(employees: List[Dict[str, Any]]) -> Dict[str, List[Dict[str, Any]]]:
    """Groupe les employés par ville."""
    # TODO: Implémenter
    pass


def filter_seniors(employees: List[Dict[str, Any]], 
                   age_threshold: int = SENIOR_AGE_THRESHOLD) -> List[Dict[str, Any]]:
    """Filtre les employés seniors (âge >= seuil)."""
    # TODO: Implémenter
    pass


def get_top_earners(employees: List[Dict[str, Any]], 
                    limit: int = TOP_EARNERS_LIMIT) -> List[Dict[str, Any]]:
    """Retourne les employés avec les salaires les plus élevés."""
    # TODO: Implémenter
    pass


# Export
def export_report(report: Dict[str, Any], output_path: str) -> None:
    """Exporte le rapport en JSON."""
    # TODO: Implémenter avec gestion d'erreurs
    pass


# Fonction principale
def process_employee_data(input_file: str, output_file: str, file_type: str = "csv") -> Dict[str, Any]:
    """Traite les données employés et génère un rapport.
    
    Args:
        input_file: Chemin vers le fichier d'entrée.
        output_file: Chemin vers le fichier de sortie.
        file_type: Type de fichier ('csv' ou 'json').
        
    Returns:
        Dictionnaire contenant le rapport d'analyse.
        
    Raises:
        EmployeeDataError: En cas d'erreur de traitement.
    """
    logger.info("Starting employee data processing...")
    
    # TODO: Implémenter en utilisant toutes les fonctions ci-dessus
    
    logger.info("Processing completed successfully")
    return report


if __name__ == "__main__":
    process_employee_data("employees.csv", "report.json", "csv")

---

## 💡 Template des tests

In [ ]:
%%writefile test_employee_analyzer.py
import pytest
import json
from pathlib import Path
# TODO: Importer vos fonctions et exceptions


@pytest.fixture
def sample_employees():
    """Fixture avec des données employés de test."""
    return [
        {"name": "Alice", "age": 28, "city": "Paris", "salary": 55000.50},
        {"name": "Bob", "age": 52, "city": "Lyon", "salary": 75000.00},
        {"name": "Charlie", "age": 35, "city": "Paris", "salary": 62000.00},
    ]


def test_calculate_average_age(sample_employees):
    """Test le calcul de l'âge moyen."""
    # TODO: Implémenter
    pass


def test_calculate_average_age_empty_list():
    """Test le calcul de l'âge moyen avec liste vide."""
    # TODO: Vérifier qu'une exception est levée
    pass


def test_filter_seniors(sample_employees):
    """Test le filtrage des seniors."""
    # TODO: Implémenter
    pass


def test_group_by_city(sample_employees):
    """Test le groupement par ville."""
    # TODO: Implémenter
    pass


def test_clean_employee_record():
    """Test le nettoyage d'un enregistrement employé."""
    # TODO: Tester avec espaces, valeurs vides, etc.
    pass


def test_read_csv_file_not_found():
    """Test la lecture d'un fichier CSV inexistant."""
    # TODO: Vérifier que FileReadError est levée
    with pytest.raises(FileReadError):
        pass


def test_get_top_earners(sample_employees):
    """Test l'obtention des meilleurs salaires."""
    # TODO: Implémenter
    pass


def test_export_report(tmp_path):
    """Test l'export du rapport."""
    # TODO: Utiliser tmp_path pour créer un fichier temporaire
    pass

---

## 🚀 Étapes recommandées

### 1. Analyse du code existant
- Lire et comprendre le code legacy
- Identifier tous les problèmes
- Lister les fonctionnalités à préserver

### 2. Créer la structure
- Définir les exceptions custom
- Définir les constantes
- Créer les signatures de fonctions

### 3. Refactoring progressif
- Commencer par les fonctions de lecture
- Puis les fonctions de nettoyage
- Puis les fonctions de calcul
- Enfin, assembler le tout dans `process_employee_data()`

### 4. Ajouter les tests au fur et à mesure
- Tester chaque fonction après l'avoir écrite
- Vérifier le coverage régulièrement

### 5. Configuration qualité
- Créer `pyproject.toml`
- Créer `.pre-commit-config.yaml`
- Installer et tester les hooks

### 6. Documentation
- Ajouter toutes les docstrings
- Créer le README.md

---

## 🧪 Commandes utiles

In [ ]:
# Installer pre-commit
# !pip install pre-commit
# !pre-commit install

# Lancer ruff
# !ruff check .
# !ruff format .

# Lancer mypy
# !mypy employee_analyzer.py

# Lancer les tests avec coverage
# !pytest --cov=employee_analyzer --cov-report=html

# Voir le rapport de coverage
# !open htmlcov/index.html

---

## 💡 Conseils pour le refactoring

### Nommage
- ❌ `data`, `data2`, `tmp`, `x`, `y`
- ✅ `employees`, `cleaned_employees`, `file_handle`, `reader`

### Longueur des fonctions
- Une fonction = une responsabilité
- Maximum 30 lignes par fonction
- Si > 30 lignes, découper

### Magic numbers
- ❌ `if d.get('age', 0) >= 50:`
- ✅ `SENIOR_AGE_THRESHOLD = 50` puis `if age >= SENIOR_AGE_THRESHOLD:`

### Context managers
- ❌ `f = open(...); ...; f.close()`
- ✅ `with open(...) as f: ...`

### Compréhensions de listes
```python
# OK pour des cas simples
ages = [emp['age'] for emp in employees if 'age' in emp]

# Mais préférer une fonction pour des cas complexes
def extract_ages(employees: List[Dict]) -> List[int]:
    """Extrait les âges des employés."""
    return [emp['age'] for emp in employees if 'age' in emp]
```

---

## 📚 Ressources

### Documentation
- [PEP 8 – Style Guide](https://peps.python.org/pep-0008/)
- [PEP 484 – Type Hints](https://peps.python.org/pep-0484/)
- [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html)
- [Ruff documentation](https://docs.astral.sh/ruff/)
- [pytest documentation](https://docs.pytest.org/)
- [pre-commit documentation](https://pre-commit.com/)

### Outils
- ruff : Linter et formatter ultra-rapide
- mypy : Type checker
- pytest : Framework de tests
- pytest-cov : Coverage pour pytest
- pre-commit : Hooks Git

---

## ✅ Critères de validation finale

Avant de soumettre, vérifiez que :

1. ✅ `ruff check .` passe sans erreur
2. ✅ `ruff format .` ne modifie rien (code déjà formaté)
3. ✅ `mypy employee_analyzer.py` passe sans erreur
4. ✅ `pytest --cov` affiche coverage > 80%
5. ✅ Tous les tests passent
6. ✅ Les hooks pre-commit sont installés
7. ✅ Le code fonctionne avec les fichiers de test
8. ✅ Toutes les fonctions ont des docstrings
9. ✅ Toutes les fonctions ont des type hints
10. ✅ Le README.md explique comment utiliser le code

**Bon courage ! 🚀**